In [49]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
import requests
import csv
import time
import pyarrow.parquet as pq

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Start a webdriver session
driver = webdriver.Firefox()

# List to store all product information
all_data = []

# Loop through two pages of results
for page in range(0, 2):
    # Navigate to page
    driver.get(f'https://www.amazon.fr/s?i=computers&rh=n%3A429879031&fs=true&page=2&qid=1675780683&ref=sr_pg_{page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'a-link-emphasis')
        cookie_button.click()
    except:
        # Get product links from the page
        product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]'))]

        # Loop through product links
        for link in product_links:
            # Navigate to product page
            driver.get(link)
            time.sleep(1)

            # Extract product information
            product = driver.find_elements(By.ID,"dp-container")[0]

            # Helper function to extract data
            def extract_data(xpath):
                try:
                    data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
                except:
                    data = " "
                return data

            # Basic product information
            name = extract_data('//span[@class="a-size-large product-title-word-break"]')
            price = extract_data('//span[@class="a-price-whole"]')
            rating = extract_data('//span[@class="a-icon-alt"]')

            # Product details in a table
            try:
                
                print("test 1")
                tbody = product.find_element(By.XPATH,'//table[@class="a-keyvalue prodDetTable"]')
                print("test 1.5")
                tr_elements = tbody.find_elements(By.XPATH, 'tr')
                print(len(tbody))
            # Dictionary to store product details
                product_details = {}
                for tr in tr_elements:
                    test("Test 2")
                    th = tr.find_element('.//th[@class="a-color-secondary a-size-base prodDetSectionEntry"]').get_attribute('textContent')
                    td = tr.find_element('.//td[@class="a-size-base prodDetAttrValue"]').get_attribute('textContent')
                    product_details[th] = td
                    print("test 3")
            except:
                pass
            # Add all product information to the list
            all_data.append({"Name": name, "Price": price, "Rating": rating, **product_details})

# Convert list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_data)

# Store DataFrame in a Parquet file
df.to_parquet('dfAmazon.parquet')

# Close the webdriver session
driver.quit()

38
['https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDExMTEwMzEzODU5ODo6MDo6&url=%2FHP-Chromebook-14c-cc0000sf-Tactile-i3-11xxx%2Fdp%2FB095ND633K%2Fref%3Dsr_1_25_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-25-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1', 'https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDAxNDQ0NjY2NTg5ODo6MDo6&url=%2FJumper-Ordinateur-Portable-bi-Bande-lextension%2Fdp%2FB09D7B1JJ4%2Fref%3Dsr_1_26_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-26-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1', 'https://www.amazon.fr/sspa/click?ie=UTF8&spc=MToyNTAyNTAwMjQzNjM4OTI0OjE2NzU4MDE2NDc6c3BfYXRmX2Jyb3dzZToyMDA2MTQ5MDQyNTE5ODo6MDo6&url=%2FOrdinateur-Portable-3000x2000-r%25C3%25A9tro%25C3%25A9clair%25C3%25A9-empreinte%2Fdp%2FB09YYDKQK5%2Fref%3Dsr_1_27_sspa%3Fqid%3D1675801647%26s%3Dcomputers%26sr%3D1-27-spons%26sp_csd%3Dd2lkZ2V0TmFt

TypeError: 'WebElement' object is not iterable

In [22]:
df = pd.read_parquet('dfAmazon.parquet', engine='pyarrow')
df

,Name,Price,Rating
0,HP Chromebook x360 14c-cc0000sf Ordina...,"689,","4,6 sur 5 étoiles"
1,"Jumper Ordinateur Portable 13,3 Pouces...","269,","4,1 sur 5 étoiles"
2,"Ordinateur Portable Windows 11, 13,5 p...","309,","3,1 sur 5 étoiles"
3,HP Laptop 17-cp0001sf PC Portable 17.3...,"899,","4,4 sur 5 étoiles"
4,HUAWEI MateBook D15 PC 15.6 pouces FHD...,"569,","4,3 sur 5 étoiles"
5,Acer Aspire 3 A314-35-P6EC Ordinateur ...,"299,","3,9 sur 5 étoiles"
6,Acer Chromebook 315 CB315-3H-C87Z Ordi...,"299,","4,3 sur 5 étoiles"
7,Aocwei Ordinateur Portable 128Go SSD S...,"223,","4,4 sur 5 étoiles"
8,CHUWI Ordinateur Portable Herobook Pro...,"239,","3,9 sur 5 étoiles"
9,TECLAST Ordinateur Portable F7Plus2 No...,"254,","4,0 sur 5 étoiles"


In [6]:
df.head()


""


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Start a webdriver session
driver = webdriver.Firefox()

# List to store all product information
all_data = []

# Loop through two pages of results
for page in range(0, 2):
    # Navigate to page
    driver.get(f'https://www.amazon.fr/s?i=computers&rh=n%3A429879031&fs=true&page=2&qid=1675780683&ref=sr_pg_{page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'a-link-emphasis')
        cookie_button.click()
    except:
        # Get product links from the page
        product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]'))]

        # Loop through product links
        for link in product_links:
            # Navigate to product page
            driver.get(link)
            time.sleep(1)

            # Extract product information
            product = driver.find_elements(By.ID,"dp-container")[0]

            # Helper function to extract data
            def extract_data(xpath):
                try:
                    data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
                except:
                    data = " "
                return data

            # Basic product information
            name = extract_data('//span[@class="a-size-large product-title-word-break"]')
            price = extract_data('//span[@class="a-price-whole"]')
            rating = extract_data('//span[@class="a-icon-alt"]')

            # Product details in a table
            try:
                print("test 0")
                WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//table[@class="a-keyvalue prodDetTable"]/tbody/tr')))
            except TimeoutException:
                 print("TableNotFound")
         
            else:
                tablerows=len(driver.find_elements(By.XPATH,'//table[@class="a-keyvalue prodDetTable"]/tbody/tr'))
            try:
                
                print("test 1")
                
                print(len(tbody))
            # Dictionary to store product details
                product_details = {}
                for tr in  range(tablerows) :
                    test("Test 2")
                    th = driver.find_element(By.XPATH,'//table[@class="a-keyvalue prodDetTable"]/tbody/tr['+str(tr+1)+']/th').get_attribute('textContent')
                    td = driver.find_element(By.XPATH,'//table[@class="a-keyvalue prodDetTable"]/tbody/tr['+str(tr+1)+']/td').get_attribute('textContent')
                    product_details[th] = td
                    print("test 3")
            except:
                pass
            # Add all product information to the list
            all_data.append({"Name": name, "Price": price, "Rating": rating, **product_details})

# Convert list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_data)

# Store DataFrame in a Parquet file
df.to_parquet('dfAmazon.parquet')

# Close the webdriver session
driver.quit()

test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
TableNotFound
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1
test 0
test 1


In [ ]:
### Must understand why i cant loop the table